In [1]:
import add_path
import datasets
import random
import json

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models.BERT.tucoregcn_bert_pytorch_processor import DialogRE
dev = DialogRE()
gen = dev._generate_examples("../datasets/DialogRE/dev.json", "dev")


In [3]:
test = gen.send(None)[1]
test

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


SpeakerRelation(speaker_x='Estelle', speaker_y='Speaker 1', rid=[7])
Speaker 1, {speaker_y} Hey!
Speaker 1, Speaker 2: Hey.
Speaker 1, Speaker 3: Hey, man. What's up?
Speaker 1, {speaker_y} Maybe you can tell me. My agent would like to know why I didn't show up at the audition I didn't know I had today. The first good thing she gets me in weeks. How could you not give me the message?!
Speaker 1, Speaker 3: Well, I'll tell ya I do enjoy guilt, but, ah, it wasn't me.
Speaker 1, Speaker 2: Yes, it was! It was him! Uh huh! Okay, it was me!
Speaker 1, {speaker_y} How is it you?
Speaker 1, Speaker 2: Well, it was just, it was all so crazy, you know. I mean, Chandler was in the closet, counting to 10, and he was up to 7 and I hadn't found a place to hide yet. I-I-I meant to tell you, and I wrote it all down on my hand. See, all of it.
Speaker 1, {speaker_y} Yep, that's my audition.
Speaker 1, Speaker 4: See, now this is why I keep notepads everywhere.
Speaker 1, Speaker 2: Yep, and that's why

{'dialog': "{speaker_y} hey!\n{speaker_2} hey.\n{speaker_3} hey, man. what's up?\n{speaker_y} maybe you can tell me. my agent would like to know why i didn't show up at the audition i didn't know i had today. the first good thing she gets me in weeks. how could you not give me the message?!\n{speaker_3} well, i'll tell ya i do enjoy guilt, but, ah, it wasn't me.\n{speaker_2} yes, it was! it was him! uh huh! okay, it was me!\n{speaker_y} how is it you?\n{speaker_2} well, it was just, it was all so crazy, you know. i mean, chandler was in the closet, counting to 10, and he was up to 7 and i hadn't found a place to hide yet. i-i-i meant to tell you, and i wrote it all down on my hand. see, all of it.\n{speaker_y} yep, that's my audition.\n{speaker_4} see, now this is why i keep notepads everywhere.\n{speaker_2} yep, and that's why we don't invite you to play.\n{speaker_5} what is the great tragedy here? you go get yourself another appointment.\n{speaker_y} well, estelle tried, you know. t

In [4]:
from models.BERT.speaker_tokens import SPEAKER_TOKENS, SpeakerBertTokenizer

n_class = 36
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
speaker_tokens = [entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]
sequence = speaker_tokenizer.tokenize(test['dialog'])
speaker_x = speaker_tokenizer.tokenize(test['relation'].speaker_x)
speaker_y = speaker_tokenizer.tokenize(test['relation'].speaker_y)

tokens = ["[CLS]"] + sequence + ["[SEP]"] + speaker_x + ["[SEP]"] + speaker_y + ["[SEP]"]
input_ids = speaker_tokenizer.convert_tokens_to_ids(tokens)
input_mask = [1] + [1]*len(sequence) + [1] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [0]
segment_ids = [0] + [0]*len(sequence) + [0] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [1]

input_speaker_ids = []
input_mention_ids = []
current_speaker_id = 0
current_speaker_idx = 0
for token in sequence:
	if SPEAKER_TOKENS.is_speaker(token):
		current_speaker_id = SPEAKER_TOKENS.convert_speaker_to_id(token)
		current_speaker_idx+=1
	input_speaker_ids.append(current_speaker_id)
	input_mention_ids.append(current_speaker_idx)
speaker_ids = [0] + input_speaker_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
mention_ids = [0] + input_mention_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]

label_id = []
for k in range(n_class):
	if k+1 in test['relation'].rid:
		label_id.append(1)
	else:
		label_id.append(0)

print(f"tokens      length: {len(tokens)}, "+" ".join([str(token) for token in tokens]))
print(f"input_ids   length: {len(input_ids)}, "+" ".join([str(input) for input in input_ids]))
print(f"input_mask  length: {len(input_mask)}, "+" ".join([str(mask) for mask in input_mask]))
print(f"segment_ids length: {len(segment_ids)}, "+" ".join([str(segment) for segment in segment_ids]))
print(f"speaker_ids length: {len(speaker_ids)}, "+" ".join([str(speaker) for speaker in speaker_ids]))
print(f"mention_ids length: {len(mention_ids)}, "+" ".join([str(mention) for mention in mention_ids]))
print(f"label_id: {label_id}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


tokens      length: 487, [CLS] {speaker_y} hey ! {speaker_2} hey . {speaker_3} hey , man . what ' s up ? {speaker_y} maybe you can tell me . my agent would like to know why i didn ' t show up at the audition i didn ' t know i had today . the first good thing she gets me in weeks . how could you not give me the message ? ! {speaker_3} well , i ' ll tell ya i do enjoy guilt , but , ah , it wasn ' t me . {speaker_2} yes , it was ! it was him ! uh huh ! okay , it was me ! {speaker_y} how is it you ? {speaker_2} well , it was just , it was all so crazy , you know . i mean , chandler was in the closet , counting to 10 , and he was up to 7 and i hadn ' t found a place to hide yet . i - i - i meant to tell you , and i wrote it all down on my hand . see , all of it . {speaker_y} yep , that ' s my audition . {speaker_4} see , now this is why i keep note ##pad ##s everywhere . {speaker_2} yep , and that ' s why we don ' t invite you to play . {speaker_5} what is the great tragedy here ? you go ge

In [5]:
from models.BERT.speaker_tokens import SPEAKER_TOKENS, SpeakerBertTokenizer
from models.BERT.tucoregcn_bert_pytorch_processor import Conversation, SpeakerRelation, Message

n_class = 36
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
c = Conversation(
	messages=[
		Message("Speaker 1", "Howdy! I'm Flowey, Flowey the Flower!"),
		Message("Speaker 2", "Hello Flowey."),
	],
	speaker_relations=[
		SpeakerRelation("Speaker 1", "Speaker 2", [3])
	]
)
test2 = c.build_inputs(speaker_tokenizer)[0]

speaker_tokens = [entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]
sequence = speaker_tokenizer.tokenize(test2['dialog'])
speaker_x = speaker_tokenizer.tokenize(test2['relation'].speaker_x)
speaker_y = speaker_tokenizer.tokenize(test2['relation'].speaker_y)

tokens = ["[CLS]"] + sequence + ["[SEP]"] + speaker_x + ["[SEP]"] + speaker_y + ["[SEP]"]
input_ids = speaker_tokenizer.convert_tokens_to_ids(tokens)
input_mask = [1] + [1]*len(sequence) + [1] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [0]
segment_ids = [0] + [0]*len(sequence) + [0] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [1]

input_speaker_ids = []
input_mention_ids = []
current_speaker_id = 0
current_speaker_idx = 0
for token in sequence:
	if SPEAKER_TOKENS.is_speaker(token):
		current_speaker_id = SPEAKER_TOKENS.convert_speaker_to_id(token)
		current_speaker_idx+=1
	input_speaker_ids.append(current_speaker_id)
	input_mention_ids.append(current_speaker_idx)
speaker_ids = [0] + input_speaker_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
mention_ids = [0] + input_mention_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]

label_id = []
for k in range(n_class):
	if k+1 in test2['relation'].rid:
		label_id.append(1)
	else:
		label_id.append(0)

print(f"tokens      length: {len(tokens)}, "+" ".join([str(token) for token in tokens]))
print(f"input_ids   length: {len(input_ids)}, "+" ".join([str(input) for input in input_ids]))
print(f"input_mask  length: {len(input_mask)}, "+" ".join([str(mask) for mask in input_mask]))
print(f"segment_ids length: {len(segment_ids)}, "+" ".join([str(segment) for segment in segment_ids]))
print(f"speaker_ids length: {len(speaker_ids)}, "+" ".join([str(speaker) for speaker in speaker_ids]))
print(f"mention_ids length: {len(mention_ids)}, "+" ".join([str(mention) for mention in mention_ids]))
print(f"label_id: {label_id}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


Speaker 1, {speaker_x} Howdy! I'm Flowey, Flowey the Flower!
Speaker 2, {speaker_x} Howdy! I'm Flowey, Flowey the Flower!
Speaker 1, Speaker 2: Hello Flowey.
Speaker 2, {speaker_y} Hello Flowey.
tokens      length: 26, [CLS] {speaker_x} how ##dy ! i ' m flow ##ey , flow ##ey the flower ! {speaker_y} hello flow ##ey . [SEP] {speaker_x} [SEP] {speaker_y} [SEP]
input_ids   length: 26, 101 30531 2129 5149 999 1045 1005 1049 4834 3240 1010 4834 3240 1996 6546 999 30525 7592 4834 3240 1012 102 30531 102 30525 102
input_mask  length: 26, 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
segment_ids length: 26, 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
speaker_ids length: 26, 0 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 10 10 10 10 10 0 0 0 0 0
mention_ids length: 26, 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 0 0 0 0 0
label_id: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
test2['dialog']

"{speaker_x} howdy! i'm flowey, flowey the flower!\n{speaker_y} hello flowey."

In [7]:
test2['relation']

SpeakerRelation(speaker_x='{speaker_x}', speaker_y='{speaker_y}', rid=[3])

In [8]:
[entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not (x[1][0].startswith("__") or type(x[1][1])!=str) else None, enumerate(SPEAKER_TOKENS.__dict__.items())))].index("{speaker_1}")

0

In [9]:
type(list(SPEAKER_TOKENS.__dict__.items())[12][1])

function